# Chapter 3 word2vec

## 3.1 추론 기반 기법과 신경망

### 3.1.1 통계 기반 기법의 문제점

- 대규모 말뭉치를 다룰 때 문제 발생, 어휘가 100만 개라면, 100만 X 100만의 행렬을 만들어야 한다.
- SVD를 적용할 때도 $O(n^3)$의 시간이 들어 비효율적이다.\

하지만 추론 기반에서는 신경망을 기반으로 미니배치로 여러 번 학습시켜 가중치를 갱신할 수 있다. 큰 작업을 처리하기 어려운 경우 나눠서 처리할 수 있다. 여러 머신과 GPU를 활용하면 속도는 더욱 빨라진다.

### 3.1.2 추론 기반 기법 개요

주변 단어(맥락)이 주어졌을 때 어떤 단어가 들어갈지 추론하는 방식이다. 이 작업을 반복하면서 단어의 출현 패턴을 학습한다.

<img src="./images/fig 3-3.png" width=500>

- 모델: 신경망
- 학습 목표: 말뭉치를 통한 올바른 추측
- 학습 결과: 단어의 분산 표현

통계 기반 기법처럼 분포 가설에 기초한다. 단어의 동시발생 가능성을 얼마나 잘 모델링하는지가 중요하다.

### 3.1.3 신경망에서의 단어 처리

대표적으로 단어를 **원핫 벡터**로 변환해서 사용한다.

단어를 원핫 벡터로 표현하는 방법:  
총 어휘 수만큼의 원소를 갖는 벡터를 준비하고, 인덱스와 같은 단어 ID의 원소를 1, 나머지는 0으로 한다. 고정 길이이기 때문에 신경망의 입력층에서 뉴런의 수를 고정할 수 있다.

<img src="./images/fig 3-7.png" width=600>

여기서는 편향을 생략

In [1]:
import numpy as np

c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
h = np.matmul(c, W)
print(h)

[[-0.33058015 -1.56249589 -0.36852109]]


원핫 벡터로 표현되어 있기 때문에 가중치로부터 행벡터를 뽑아내는 것으로 볼 수 있다.

In [2]:
import numpy as np
from common.layers import MatMul

c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[-0.46016462  0.4332086  -1.10626672]]


## 3.2 단순한 word2vec

#### word2vec
- CBOW(continuous bag-of-words) 모델
- skip-gram

### 3.2.1 CBOW 모델의 추론 처리

- 타깃: 중앙 단어
- 맥락: 주변 단어
- 맥락으로부터 타깃을 추측하는 신경망
- 신경망의 입력: 맥락

<img src="./images/fig 3-9.png" width=600>

- 입력층: 2개
    - 맥락으로 고려할 단어 수로 입력층의 수가 정해진다.
- $W_{in}$: 은닉층으로 변환시키는 가중치, 모든 입력층에 똑같이 적용
    - 각 행에 해당 단어의 분산 표현이 담겨 있다.
    - 학습을 진행할수록 단어를 잘 추측하는 방향으로 표현이 갱신됨
- $W_{out}$: 출력층으로 변환시키는 가중치
- 은닉층의 뉴런은 변환된 값들의 평균
- 출력층은 각 단어별 점수이고 소프트맥스 함수로 확률을 얻을 수 있다.
- 은닉층의 뉴런 수를 입력층의 뉴런 수보다 적게 하는 것이 중요하다. 
    - 단어 예측에 필요한 정보를 간결하게 담게 되며, 밀집벡터 표현을 얻을 수 있기 때문이다.
    - 인코딩: 입력 -> 은닉(인간 이해 X)
    - 디코딩: 은닉 -> 출력(인간 이해할 수 있도록 복원)
    
<img src="./images/fig 3-11.png" width=600>

In [3]:
# CBOW 모델의 추론 과정
import numpy as np
from common.layers import MatMul

# 샘플 맥락 데이터
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

# 가중치 초기화
W_in = np.random.randn(7, 3)    # 두 맥락은 동일한 W_in을 공유한다
W_out = np.random.randn(3, 7)

# 계층 생성
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# 순전파
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

print(s)

[[-1.74801686  0.27589949  0.26169741 -1.49295695 -1.2378314  -1.53780328
  -4.46044015]]


주의할 점: 입력층이 여러 개 있고, 가중치를 공유한다.

### 3.2.2 CBOW 모델의 학습

<img src="./images/fig 3-14.png">

- 다중 클래스 분류라 softmax 함수를 사용해서 점수를 확률로 반환하고 교차 엔트로피 오차로 손실을 계산한다.

### 3.2.3 word2vec의 가중치와 분산 표현

가중치에는 단어의 분산 표현이 저장돼있다.
- 입력 측 가중치: 행이 각 단어의 분산 표현에 해당
- 출력 측 가중치: 열이 각 단어의 분산 표현에 해당

최종적으로 입력 측 가중치를 단어의 분산 표현으로 선택하는 것이 대중적이다.

## 3.3 학습 데이터 준비

### 3.3.1 맥락과 타깃

- 맥락: 주변 단어, 여러 개가 될 수 있다.
- 타깃: 목표로 하는 단어, 단 1개
- 맥락을 통해 타깃이 나올 확률을 높이는 방향으로 학습

In [5]:
from common.util import preprocess

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)
print(id_to_word)

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [6]:
# corpus를 주면 맥락과 타깃으로 나눠주는 함수
def create_contexts_target(corpus, window_size=1):
    target = corpus[window_size:-window_size]
    contexts = []
    
    for idx in range(window_size, len(corpus)-window_size):
        cs = []
        for t in range(-window_size, window_size+1):
            if t == 0:
                continue
            cs.append(corpus[idx+t])
        contexts.append(cs)
        
    return np.array(contexts), np.array(target)

In [7]:
contexts, target = create_contexts_target(corpus, window_size=1)
print(contexts)
print(target)

[[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]]
[1 2 3 4 1 5]


### 3.3.2 원핫 표현으로 변환

In [8]:
# 단어 ID를 원핫 표현으로 변환하는 함수 구현
from common.util import preprocess, create_contexts_target, convert_one_hot

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

contexts, target = create_contexts_target(corpus, window_size=1)

vocab_size = len(word_to_id)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

In [9]:
print(target)
print(contexts)

[[0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0]]
[[[1 0 0 0 0 0 0]
  [0 0 1 0 0 0 0]]

 [[0 1 0 0 0 0 0]
  [0 0 0 1 0 0 0]]

 [[0 0 1 0 0 0 0]
  [0 0 0 0 1 0 0]]

 [[0 0 0 1 0 0 0]
  [0 1 0 0 0 0 0]]

 [[0 0 0 0 1 0 0]
  [0 0 0 0 0 1 0]]

 [[0 1 0 0 0 0 0]
  [0 0 0 0 0 0 1]]]


In [10]:
print(target.shape)
print(contexts.shape)

(6, 7)
(6, 2, 7)
